## Goals of Project

1. Automate the process of writing notes in a natural language format of each tract in a BLM Oil and Gas Lease Sale

2.  Create visualizaitons tract by tract for CEO depicting permitting, new production, and leasing activity within a given radius

3.  Create a predictive model to estimate the purchase price based on historical activity, production, permits, and commodity prices

## Goals of Notebook

1.  Read in data necessary for sale note generation
2.  Clean dataframes by dropping uneeded features
3.  Convert date related columns to datetime.  
4.  Convert coordinate systems for geodata 
5.  Write function to filter data in defined mile radius for each tract in sale
6.  Export filtered data for sale note generationa

In [2]:
import pandas as pd
import geopandas as gp
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
import shapely
import math
import datetime as dt
import pdb

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
#had to add GDAL_DATA variable to system variables and set value to the folder of gdal in C:\Users\mishaun\AppData\Local\Continuum\anaconda3\Library\share\gdal on my work computer
'GDAL_DATA' in os.environ

False

# Reading in Data/Data Preparation for Automating Sale Notes

In [3]:
shapezipfile = ("zip://Data/BLM UT Sep 29, 2020 Shapefile.zip")
tractshp = gp.read_file(shapezipfile, encoding = "utf-8")

In [4]:
#this is shapefile downloaded from drillinginfo holding well information
prodshp = gp.read_file("zip://Data/production.ZIP")
permitshp = gp.read_file("zip://Data/permits.ZIP")

#reading in csv of leases - converting to a GeoDataFrame - initial coord system is epsg:4326
leases = pd.read_csv("Data/LeasesTable.CSV")
leasesgeo = gp.GeoDataFrame(leases, crs = {'init': 'epsg:4326'}, geometry=gp.points_from_xy(leases["Longitude (WGS84)"], leases["Latitude (WGS84)"]))

oldprod = gp.read_file("zip://data/oldProduction.zip")

In [5]:
saleDate = pd.Timestamp(dt.date(2020, 3, 23))

### Trimming and Cleaning Data

In [6]:
leasesgeo.columns

Index(['State/Province', 'Effective Date', 'Record Date',
       'Expiration of Primary Term', 'Term (Months)', 'Grantor',
       'Grantee Alias', 'Royalty', 'Bonus', 'Area (Acres)', 'Section',
       'Township', 'Township Direction', 'Range', 'Range Direction',
       'Vol/Page', 'Record Number', 'Instrument Type', 'Instrument Date',
       'County/Parish', 'Options/Extensions', 'DI Basin', 'Ext. Bonus',
       'Ext. Term (Months)', 'Abstract', 'Block', 'BLM', 'State Lease',
       'Grantee', 'Max Depth', 'Majority Legal Assignee', 'DI Subplay',
       'Min Depth', 'Majority Assignment Effective Date', 'Latitude (WGS84)',
       'DI Play', 'Majority Legal Assignee Interest', 'Longitude (WGS84)',
       'Majority Assignment Vol/Page', 'geometry'],
      dtype='object')

In [9]:

leasesgeo.drop(columns = ['Instrument Type', 'Instrument Date','Options/Extensions', 'DI Basin', 'Ext. Bonus',
       'Ext. Term (Months)', 'Abstract', 'Block', 'BLM', 'State Lease',
       'Grantee', 'Grantor Address', 'Grantee Address', 'Max Depth',
       'Majority Legal Assignee', 'DI Subplay', 'Min Depth',
       'Majority Assignment Effective Date','Majority Legal Assignee Interest','Majority Assignment Vol/Page'], inplace = True)

KeyError: "['Grantor Address' 'Grantee Address'] not found in axis"

In [10]:
prodshp.columns

Index(['APIUWI', 'OpAlias', 'OpCompany', 'OpReported', 'OpTicker', 'LeaseName',
       'WellNo', 'EntityType', 'County', 'DIBasin',
       ...
       'PGC_Area', 'Latitude', 'Longitude', 'Last12Liq', 'Last12Gas',
       'Last12Wtr', 'EntityId', 'OtherNo', 'OpAliasID', 'geometry'],
      dtype='object', length=112)

In [11]:
prodshp.drop(columns = ['LatestWtr','CumWtr',
       'Prior12Liq', 'Prior12Gas', 'LastTestDt', 'Prior12Wtr', 'LastFlwPrs',
       'LastWHSIP', '2moGOR', 'LatestGOR', 'CumGOR', 'Lst12Yield', '2moYield',
       'LatestYld', 'PeakGas', 'PkGasMoNo', 'PeakLiq', 'PkLiqMoNo', 'PeakBOE',
       'PkBOEMoNo', 'PkMMCFGE', 'PkMMCFGMoN', 'TopPerf', 'BtmPerf', 'GasGrav',
       'OilGrav','CompDate', 'GasGather',
       'LiqGather', 'LeaseNo'], inplace = True)

In [12]:
permitshp.columns

Index(['API10UWI', 'WGID', 'PermitNo', 'Country', 'OFS_Reg', 'State', 'County',
       'DIBasin', 'DIPlay', 'DISubplay', 'District', 'OpAlias', 'OpCompany',
       'OpReported', 'OpTicker', 'LeaseName', 'WellNo', 'LeaseNo', 'Field',
       'Formation', 'PermitType', 'WellType', 'DrillType', 'PermDepth',
       'PermDUOM', 'TVD', 'TVD_UOM', 'PermStatus', 'WellStatus', 'FiledDate',
       'AprvdDate', 'AmendDate', 'CntctName', 'CntctPhone', 'OperAddrs',
       'OperCity', 'OperState', 'OperZip', 'OperCity30', 'OperCity50',
       'Section', 'Township', 'Range', 'Block', 'Survey', 'Abstract',
       'H2S_Area', 'Latitude', 'Longitude', 'Btm_Lat', 'Btm_Lon', 'ExpDate',
       'geometry'],
      dtype='object')

In [13]:
permitshp.drop(columns = ['OpReported', 'AmendDate', 'CntctName',
       'CntctPhone', 'OperAddrs', 'OperCity', 'OperState', 'OperZip',
       'OperCity30', 'Section', 'OperCity50', 'Township', 'Range', 'Block',
       'Survey', 'TVD_UOM', 'Abstract', 'WGID', 'H2S_Area','OFS_Reg', 'LeaseNo', 'PermDUOM',
       'PermitNo','OpCompany', 'OpTicker'], inplace=True)

In [14]:
oldprod.info()
oldprod.columns

dropcols = ['CumWtr', 'DISubplay', '1moLiq',
       '1moGas', '6moLiq', 'DIBasin', '6moGas', '6moBOE', '6moWater', 'DIPlay',
       'PracIP_Liq', 'PracIP_BOE', 'PracIP_Gas', 'PrcIPCFGED', 'LatestWtr',
       'Prior12Liq', 'Prior12Gas', 'LastTestDt', 'Prior12Wtr', 'LastFlwPrs',
       'LastWHSIP', '2moGOR', 'LatestGOR', 'CumGOR', 'Lst12Yield', '2moYield',
       'LatestYld', 'PeakGas', 'PkGasMoNo', 'PeakLiq', 'PkLiqMoNo', 'PeakBOE',
       'PkBOEMoNo', 'PkMMCFGE', 'PkMMCFGMoN', 'TopPerf', 'BtmPerf', 'GasGrav',
       'OilGrav', 'CompDate', 'WellCount', 'MaxActvWel', 'GasGather',
       'LiqGather', 'LeaseNo', 'PerfLength', 'TVD', 'Field', 'State',
       'District', 'GeoProvin','Country','OCS_Area', 'PGC_Area',
       'OpReported', 'Survey', 'EntityId', 'Last12Liq', 'Last12Gas',
       'Last12Wtr', 'OtherNo']

oldprod.drop(columns=dropcols, inplace = True)

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 28812 entries, 0 to 28811
Columns: 112 entries, APIUWI to geometry
dtypes: float64(64), geometry(1), int64(2), object(45)
memory usage: 24.6+ MB


Index(['APIUWI', 'OpAlias', 'OpCompany', 'OpReported', 'OpTicker', 'LeaseName',
       'WellNo', 'EntityType', 'County', 'DIBasin',
       ...
       'PGC_Area', 'Latitude', 'Longitude', 'Last12Liq', 'Last12Gas',
       'Last12Wtr', 'EntityId', 'OtherNo', 'OpAliasID', 'geometry'],
      dtype='object', length=112)

In [15]:
leasesgeo.info()
prodshp.info()
permitshp.info()
oldprod.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 7783 entries, 0 to 7782
Data columns (total 40 columns):
State/Province                        7783 non-null object
Effective Date                        5350 non-null object
Record Date                           7783 non-null object
Expiration of Primary Term            7783 non-null object
Term (Months)                         7783 non-null int64
Grantor                               7783 non-null object
Grantee Alias                         6769 non-null object
Royalty                               6070 non-null float64
Bonus                                 2012 non-null float64
Area (Acres)                          7765 non-null float64
Section                               7783 non-null float64
Township                              7783 non-null float64
Township Direction                    7783 non-null object
Range                                 7783 non-null float64
Range Direction                       7783 non-null ob

### Converting geodataframes to same coord system - UTM system for creating buffers

In [16]:
def convertCRS(*args, crs_system = 26913):
    for i in args:
        print(i.crs)
        i.to_crs(epsg = crs_system, inplace = True)
        print("Converted to:")
        print(i.crs)
        print("\n")

def convertoDateTime(*dataframe):
    '''
    This function will take in arguments of a dataframe.
    It will then search each column name to see if there is a date in the name using the filter method
    The filtered list will then use the apply method to convert data in column to date time
    '''
    for df in dataframe:
        cols = list(filter(lambda x: 'date' in x.lower(), df.columns))

        for col in cols:
            df[col] = df[col].apply(lambda x: pd.to_datetime(x))

In [17]:
convertCRS(prodshp, permitshp, leasesgeo, tractshp, oldprod)

prodshp.crs
permitshp.crs
leasesgeo.crs
oldprod.crs

{'init': 'epsg:4326'}
Converted to:
{'init': 'epsg:26913', 'no_defs': True}


{'init': 'epsg:4326'}
Converted to:
{'init': 'epsg:26913', 'no_defs': True}


{'init': 'epsg:4326'}
Converted to:
{'init': 'epsg:26913', 'no_defs': True}


{'init': 'epsg:3857'}
Converted to:
{'init': 'epsg:26913', 'no_defs': True}


{'init': 'epsg:4326'}
Converted to:
{'init': 'epsg:26913', 'no_defs': True}




{'init': 'epsg:26913', 'no_defs': True}

{'init': 'epsg:26913', 'no_defs': True}

{'init': 'epsg:26913', 'no_defs': True}

{'init': 'epsg:26913', 'no_defs': True}

In [18]:
convertoDateTime(prodshp, permitshp, leasesgeo, oldprod)

### Adding a column to tract shapefile data for centroids of each tract for creating buffers upon

In [19]:
tractshp["centroids"] = tractshp.centroid
#calculating acres of tract, conv to acres is m^2 to acres
tractshp['Acres'] = round(tractshp.area * 0.000247105)
tractshp.head()

,lot_no,tract_id,short_code,label,geometry,centroids,Acres
0,70346,1,BLMUT-2020-Q3,UT-2020-09-0736,"POLYGON ((-99122.780 4401888.800, -98731.645 4...",POINT (-100102.355 4401180.076),1038.0
1,70347,2,BLMUT-2020-Q3,UT-2020-09-0739,"POLYGON ((-99037.715 4397250.637, -98628.152 4...",POINT (-99462.603 4397737.032),914.0
2,70348,3,BLMUT-2020-Q3,UT-2020-09-0956,"POLYGON ((-97778.183 4350035.776, -97806.782 4...",POINT (-99347.295 4350543.796),1299.0
3,70349,4,BLMUT-2020-Q3,UT-2020-09-0959,"POLYGON ((-97890.495 4348406.251, -97918.619 4...",POINT (-99728.358 4348388.510),1878.0
4,70350,5,BLMUT-2020-Q3,UT-2020-09-0984,"POLYGON ((-94674.765 4323070.560, -94686.090 4...",POINT (-95266.155 4324744.726),1258.0


In [20]:
#adding buffer around centroid point from tract of 3 mi (1609.34 meters = 1 mile)
miradius = 3
milesbuffer = miradius * 1609.34

tractshp["buffers"] = tractshp.centroids.apply(lambda x: x.buffer(milesbuffer,20))


# Testing Spatial Filters

In [22]:
TestT = 1
tractTest = tractshp[tractshp["tract_id"] == TestT].iloc[0]


permFiltered = permitshp.loc[permitshp.within(tractshp[tractshp["tract_id"] ==TestT]["buffers"].iloc[0])]
prodFiltered = prodshp.loc[prodshp.within(tractshp[tractshp["tract_id"] ==TestT]["buffers"].iloc[0])]
leasesFiltered = leasesgeo.loc[leasesgeo.within(tractshp[tractshp["tract_id"]==TestT]["buffers"].iloc[0])]

In [23]:
len(permFiltered), len(prodFiltered), len(leasesFiltered)

(0, 0, 13)

### Calculating distance between leases within 3 mi radius and tract's centroid - Store in Function

In [24]:
#function to get compass direction

def cardDir(point, tractRef):
    '''
    This function will take in 2 inputs and will calculate the cardinal direction between 2 points
    
    1.  point: a shapely Point object referring to a permit,lease, well found in spatial query  
    2.  tractRef: a shapely Point object referring to the centroid of the lease/tract for analysis
    '''
    
    #storing each coordinate in a temporary variable
    a = point.x
    b = point.y
    c = tractRef["centroids"].x
    d = tractRef["centroids"].y

    #south and west are positive direction for my axis convention
    western = c - a
    southern = d - b
    
    #calculating angle between points for determining which direction the point of data is in reference to tract centroid
    compdeg = abs(math.atan(western/southern))
    #converts radians to degrees
    compdeg *= 57.2958
    compdeg

    if southern > 0:
        if compdeg < 10:
            carddir = "S"
        elif compdeg > 80:
            if western>0:
                carddir = "W"
            else:
                carddir = "E"
        else:
            if western>0:
                carddir = "SW"
            else:
                carddir = "SE"
    else:
        if compdeg < 10:
            carddir = "N"
        elif compdeg > 80:
            if western>0:
                carddir = "W"
            else:
                carddir = "E"
        else:
            if western>0:
                carddir = "NW"
            else:
                carddir = "NE"  
     
    return carddir         
   
    

### Creating Function from Test Above and Below Code to Get Filtered Data Tract by Tract In a Loop

In [25]:
def prepareTractFilter(x):
    '''This function will use global variables defined previously for testing each tract
       x will be the tract number we will look for activity around
       
       The functino will run and add columns of data only necessary for the filtered values data
    '''
    
    #getting tract in sale list to search well data (permits, leases, prod)
    tractTest = tractshp[tractshp["tract_id"] == x].iloc[0]

    #global variables permitshp, prodshp, leasesgeo - not passing them into the function
    #this will use geopandas function within to get data within buffer of test tract and then use the boolean array to filter itself to get data
    permitstoeval = permitshp.loc[permitshp.within(tractTest["buffers"])]
    prodtoeval = prodshp.loc[prodshp.within(tractTest["buffers"])]
    leasestoeval = leasesgeo.loc[leasesgeo.within(tractTest["buffers"])]
    oldprodtoeval = oldprod.loc[oldprod.within(tractTest["buffers"])]
    
    #calculating distance (in miles) away lease within 3 mi radius is to tract of interest and direction of data point (permit, lease, prod)
    leasestoeval["distance"] = leasestoeval["geometry"].apply(lambda x: x.distance(tractTest["centroids"])/1609.34)
    leasestoeval["direction"] = leasestoeval["geometry"].apply(lambda x: cardDir(x, tractTest))
    leasestoeval["RecordYr"] = leasestoeval["Record Date"].apply(lambda x: x.year)
    
    
    #adding columns to permits dataframe such as horizontal lngth and direction
    permitstoeval["horzLength"] = permitstoeval["PermDepth"] - permitstoeval["TVD"]
    permitstoeval["distance"] = permitstoeval["geometry"].apply(lambda x: x.distance(tractTest["centroids"])/1609.34)
    permitstoeval["direction"] = permitstoeval["geometry"].apply(lambda x: cardDir(x, tractTest))


    prodtoeval["distance"] = prodtoeval["geometry"].apply(lambda x: x.distance(tractTest["centroids"])/1609.34)
    prodtoeval["direction"] = prodtoeval["geometry"].apply(lambda x: cardDir(x, tractTest))
    
    oldprodtoeval["distance"] = oldprodtoeval["geometry"].apply(lambda x: x.distance(tractTest["centroids"])/1609.34)
    oldprodtoeval["direction"] = oldprodtoeval["geometry"].apply(lambda x: cardDir(x, tractTest))
    

    return permitstoeval, prodtoeval, leasestoeval, oldprodtoeval

### Looping Through All Sale Tracts - Retrieving Data Around Tract

In [26]:
#output filtered data for hard storing to excel file
outputProd = []
outputPerm = []
outputLeases = []
outputOldProd = []


#looping through each tract id and creating filtered geospatial data, activity summaries, and visualizaiton plots
for i in tractshp["tract_id"]:
    perm, prod, leases, oldprodtoeval = prepareTractFilter(i)
    
    #appending geospatial filtered datasets to a list in order to write dataframes to file for easier retrieval and data validaiton
    perm["tract_id"] = i
    outputPerm.append(perm)
    
    prod["tract_id"] = i
    outputProd.append(prod)
    
    leases["tract_id"] = i
    outputLeases.append(leases)
    
    oldprodtoeval["tract_id"] = i
    outputOldProd.append(oldprodtoeval)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

### Writing Filtered Data to File for Easier Retrieval

In [27]:
    
#concatenating filtered dataframes along the row axis and writing to file
pd.concat(outputLeases).to_excel("Output Data/Sale Tracts Activity Data/Leases Around Sale Tracts.xlsx")
pd.concat(outputPerm).to_excel("Output Data/Sale Tracts Activity Data/Permits Around Sale Tracts.xlsx")
pd.concat(outputProd).to_excel("Output Data/Sale Tracts Activity Data/Prod Around Sale Tracts.xlsx")
pd.concat(outputOldProd).to_excel("Output Data/Sale Tracts Activity Data/OldProd Around Sale Tracts.xlsx")

In [28]:
# having to store geodataframes instead of exporting due to issues exporting errors from geopandas package
# storing vars puts them in a database global to all other scripts and directories on this system - thus, they will be overwritten in other sales
# This will force each notebook to be executed when jumping from sale to sale
# converted data types and multiple Shapely columns are unsupported when exporting 
# for example, tractshp has added Shapely columns like buffers and centroid
# prodshp has datetime columns that break the export

%store tractshp
%store milesbuffer
%store saleDate

%store prodshp
%store permitshp

Stored 'tractshp' (GeoDataFrame)
Stored 'milesbuffer' (float)
Stored 'saleDate' (Timestamp)
Stored 'prodshp' (GeoDataFrame)
Stored 'permitshp' (GeoDataFrame)
